# Example: SMIRNOFF99Frosst is not matching alpha-cyclodextrin bond types

In [1]:
import parmed as pmd
from openeye.oechem import *
from openforcefield.typing.engines.smirnoff import *

In [2]:
def load_mol2(filename, name=None, add_tripos=True):
    """
    Converts a `mol2` file to an `OEMol` object.
    Parameters
    ----------
    filename : str
        MOL2 file
    name : str
        Residue name
    add_tripos : bool
        Whether to add Tripos atom names to the file

    Returns
    -------
    openeye.oechem.OEMol

    """
    ifs = oemolistream()
    molecules = []
    if not ifs.open(filename):
        print(f'Unable to open {filename} for reading...')
    for mol in ifs.GetOEMols():
        if add_tripos:
            OETriposAtomNames(mol)
        if name:
            mol.SetTitle(name)
        # Add all the molecules in this file to a list, but only return the first one.
        molecules.append(OEMol(mol))
    return molecules[0]

In [3]:
def create_host_topology(components, host_resname):
    """Return the topology components belonging the host only.
    
    Parameters:
    ----------
    components : parmed.topology.components
        ParmEd topology components, split by molecule
    host_resname : str
        Residue name of the host molecule (no colon)
    Returns
    -------
    pmd.Structure
        A ParmEd structure containing just the host and guest molecule
    """
    topology = pmd.Structure()
    for component in components:
        # Check the first residue of each component becuase there may be multiple residues in each component, but they shoudl all have the same residue name.hash
        if component[0].residues[0].name == host_resname.upper():
            topology += component[0]
    return topology

In [4]:
def split_topology(file_name):
    """Split a file into component topology using ParmEd.
    
    Parameters:
    ----------
    file_name : str
        Structure file
    """

    topology = pmd.load_file(file_name)
    return topology.split()

## Try with GAFF atom types

In [5]:
host = load_mol2('generated/a-bam-p/MGO.mol2')
components = split_topology(file_name='generated/a-bam-p/full.pdb')
host_topology = create_host_topology(components, host_resname='MGO')

In [6]:
host.GetMaxAtomIdx()

126

In [7]:
host_topology

<Structure 126 atoms; 6 residues; 132 bonds; NOT parametrized>

In [8]:
ff = ForceField('forcefield/smirnoff99Frosst.ffxml') 
system = ff.createSystem(host_topology.topology, [host],
                         nonbondedCutoff=1.1*unit.nanometer, 
                         ewaldErrorTolerance=1e-4
                         )


In [9]:
host_structure = pmd.openmm.topsystem.load_topology(host_topology.topology, system, host_topology.positions)

In [10]:
# pmd.tools.printBonds(host_structure)

In [11]:
for bond in host_structure.bonds:
    atom1, atom2 = bond.atom1, bond.atom2
    if bond.type is not None:
        if bond.type.req == 4.0000:
            print('%7d %4s (%4s) %7d %4s (%4s) %10.4f %10.4f' % (
            atom1.idx+1, atom1.name, atom1.type, atom2.idx+1,
            atom2.name, atom2.type, bond.type.req, bond.type.k)
            )


      1   C1 (   1)       3   O1 (   3)     4.0000  1000.0000
      1   C1 (   1)      16   O5 (  16)     4.0000  1000.0000
      3   O1 (   3)      33   C4 (  33)     4.0000  1000.0000
      6   O2 (   6)       7  HO2 (   7)     4.0000  1000.0000
     10   O3 (  10)      11  HO3 (  11)     4.0000  1000.0000
     12   C4 (  12)     108   O1 ( 108)     4.0000  1000.0000
     14   C5 (  14)      16   O5 (  16)     4.0000  1000.0000
     20   O6 (  20)      21  HO6 (  21)     4.0000  1000.0000
     22   C1 (  22)      24   O1 (  24)     4.0000  1000.0000
     22   C1 (  22)      37   O5 (  37)     4.0000  1000.0000
     24   O1 (  24)      54   C4 (  54)     4.0000  1000.0000
     27   O2 (  27)      28  HO2 (  28)     4.0000  1000.0000
     31   O3 (  31)      32  HO3 (  32)     4.0000  1000.0000
     35   C5 (  35)      37   O5 (  37)     4.0000  1000.0000
     41   O6 (  41)      42  HO6 (  42)     4.0000  1000.0000
     43   C1 (  43)      45   O1 (  45)     4.0000  1000.0000
     43 

## Try with SYBYL atom types

In [12]:
host = load_mol2('generated/a-bam-p/MGO-sybyl.mol2')
components = split_topology(file_name='generated/a-bam-p/full.pdb')
host_topology = create_host_topology(components, host_resname='MGO')

In [13]:
host.GetMaxAtomIdx()

126

In [14]:
host_topology

<Structure 126 atoms; 6 residues; 132 bonds; NOT parametrized>

In [15]:
ff = ForceField('forcefield/smirnoff99Frosst.ffxml') 
system = ff.createSystem(host_topology.topology, [host],
                         nonbondedCutoff=1.1*unit.nanometer, 
                         ewaldErrorTolerance=1e-4
                         )


In [16]:
host_structure = pmd.openmm.topsystem.load_topology(host_topology.topology, system, host_topology.positions)

In [17]:
# pmd.tools.printBonds(host_structure)

In [18]:
for bond in host_structure.bonds:
    atom1, atom2 = bond.atom1, bond.atom2
    if bond.type is not None:
        if bond.type.req == 4.0000:
            print('%7d %4s (%4s) %7d %4s (%4s) %10.4f %10.4f' % (
            atom1.idx+1, atom1.name, atom1.type, atom2.idx+1,
            atom2.name, atom2.type, bond.type.req, bond.type.k)
            )


      1   C1 (   1)       3   O1 (   3)     4.0000  1000.0000
      1   C1 (   1)      16   O5 (  16)     4.0000  1000.0000
      3   O1 (   3)      33   C4 (  33)     4.0000  1000.0000
      6   O2 (   6)       7  HO2 (   7)     4.0000  1000.0000
     10   O3 (  10)      11  HO3 (  11)     4.0000  1000.0000
     12   C4 (  12)     108   O1 ( 108)     4.0000  1000.0000
     14   C5 (  14)      16   O5 (  16)     4.0000  1000.0000
     20   O6 (  20)      21  HO6 (  21)     4.0000  1000.0000
     22   C1 (  22)      24   O1 (  24)     4.0000  1000.0000
     22   C1 (  22)      37   O5 (  37)     4.0000  1000.0000
     24   O1 (  24)      54   C4 (  54)     4.0000  1000.0000
     27   O2 (  27)      28  HO2 (  28)     4.0000  1000.0000
     31   O3 (  31)      32  HO3 (  32)     4.0000  1000.0000
     35   C5 (  35)      37   O5 (  37)     4.0000  1000.0000
     41   O6 (  41)      42  HO6 (  42)     4.0000  1000.0000
     43   C1 (  43)      45   O1 (  45)     4.0000  1000.0000
     43 

## Visualize with `mdtraj`

In [19]:
import nglview, mdtraj

In [20]:
traj = mdtraj.load('generated/a-bam-p/MGO.mol2')

/home/dslochower/data/applications/anaconda3/envs/openforcefield/lib/python3.6/site-packages/pandas/io/parsers.py:2138: FutureWarning: split() requires a non-empty pattern match.
  yield pat.split(line.strip())
/home/dslochower/data/applications/anaconda3/envs/openforcefield/lib/python3.6/site-packages/pandas/io/parsers.py:2140: FutureWarning: split() requires a non-empty pattern match.
  yield pat.split(line.strip())


In [21]:
view = nglview.show_mdtraj(traj)
view

A Jupyter Widget

## File contents

In [22]:
!cat generated/a-bam-p/MGO.mol2

@<TRIPOS>MOLECULE
Cpptraj generated mol2 file.
  126   132     6     0     0
SMALL
USER_CHARGES


@<TRIPOS>ATOM
      1 C1         16.4460   15.0130   33.8240 c3         1 MGO      0.304529
      2 H1         16.4970   13.9160   33.9690 h2         1 MGO      0.090782
      3 O1         17.7530   15.5660   33.4500 os         1 MGO     -0.453070
      4 C2         15.4640   15.3080   32.6940 c3         1 MGO      0.092199
      5 H2         14.4850   14.8820   32.9300 h1         1 MGO      0.102926
      6 O2         15.9650   14.7370   31.4740 oh         1 MGO     -0.586639
      7 HO2        16.9180   14.5780   31.5580 ho         1 MGO      0.425065
      8 C3         15.3500   16.8010   32.4700 c3         1 MGO      0.093211
      9 H3         16.3090   17.1870   32.1150 h1         1 MGO      0.064468
     10 O3         14.3320   16.9970   31.4760 oh         1 MGO     -0.592567
     11 HO3        14.4270   16.2760   30.8380 ho         1 MGO      0.421017
     12 C4         14.9500   1

In [23]:
! cat generated/a-bam-p/MGO-sybyl.mol2

@<TRIPOS>MOLECULE
Cpptraj generated mol2 file.
  126   132     0     0     0
SMALL
USER_CHARGES

@<TRIPOS>ATOM
      1 C1         16.4460   15.0130   33.8240 C.3       1 <0>         0.3045
      2 H1         16.4970   13.9160   33.9690 H         1 <0>         0.0908
      3 Os1        17.7530   15.5660   33.4500 Os        1 <0>        -0.4531
      4 C2         15.4640   15.3080   32.6940 C.3       1 <0>         0.0922
      5 H2         14.4850   14.8820   32.9300 H         1 <0>         0.1029
      6 O1         15.9650   14.7370   31.4740 O.3       1 <0>        -0.5866
      7 Ho1        16.9180   14.5780   31.5580 Ho        1 <0>         0.4251
      8 C3         15.3500   16.8010   32.4700 C.3       1 <0>         0.0932
      9 H3         16.3090   17.1870   32.1150 H         1 <0>         0.0645
     10 O2         14.3320   16.9970   31.4760 O.3       1 <0>        -0.5926
     11 Ho2        14.4270   16.2760   30.8380 Ho        1 <0>         0.4210
     12 C4         14.9500   17

In [24]:
!cat generated/a-bam-p/full.pdb

CRYST1   38.861   39.115   56.412  90.00  90.00  90.00               1
ATOM      1  Pb  DUM     1      19.422  19.383  26.526  1.00  0.00           P  
TER       2      DUM     1 
ATOM      2  Pb  DUM     2      19.422  19.383  21.526  1.00  0.00           P  
TER       3      DUM     2 
ATOM      3  Pb  DUM     3      19.422  22.883  18.026  1.00  0.00           P  
TER       4      DUM     3 
ATOM      4  C1  MGO     4      16.446  15.013  33.824  1.00  0.00           C  
ATOM      5  H1  MGO     4      16.497  13.916  33.969  1.00  0.00           H  
ATOM      6  O1  MGO     4      17.753  15.566  33.450  1.00  0.00           O  
ATOM      7  C2  MGO     4      15.464  15.308  32.694  1.00  0.00           C  
ATOM      8  H2  MGO     4      14.485  14.882  32.930  1.00  0.00           H  
ATOM      9  O2  MGO     4      15.965  14.737  31.474  1.00  0.00           O  
ATOM     10  HO2 MGO     4      16.918  14.578  31.558  1.00  0.00           H  
ATOM     11  C3  MGO     4      15.

ATOM   4676  H1  WAT  1529       2.968  36.380   3.099  1.00  0.00           H  
ATOM   4677  H2  WAT  1529       3.418  36.624   4.524  1.00  0.00           H  
TER    4678      WAT  1529 
ATOM   4678  O   WAT  1530       4.883  21.664   2.401  1.00  0.00           O  
ATOM   4679  H1  WAT  1530       5.412  20.877   2.272  1.00  0.00           H  
ATOM   4680  H2  WAT  1530       5.515  22.343   2.634  1.00  0.00           H  
TER    4681      WAT  1530 
ATOM   4681  O   WAT  1531       7.076  21.401   5.455  1.00  0.00           O  
ATOM   4682  H1  WAT  1531       6.447  20.700   5.622  1.00  0.00           H  
ATOM   4683  H2  WAT  1531       6.537  22.162   5.239  1.00  0.00           H  
TER    4684      WAT  1531 
ATOM   4684  O   WAT  1532       7.202  20.680  17.891  1.00  0.00           O  
ATOM   4685  H1  WAT  1532       6.920  21.183  17.127  1.00  0.00           H  
ATOM   4686  H2  WAT  1532       7.584  19.882  17.525  1.00  0.00           H  
TER    4687      WAT  153